# The Advanced of Image Preprocessing

### 本章節內容大綱
* [影像二值化](#影像二值化)
* [用 cv2.findContours 找影像輪廓](#用-cv2.findContours-找影像輪廓)
* [K-Means Clustering in OpenCV](#K-Means-Clustering-in-OpenCV)

In [ ]:
# opencv 在 python 中的 module 為 cv2
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# upload aia_log.png
!gdown --id 1mXPIv85qo-lh3A7bHsuegzyWdBMuBN-H

In [ ]:
image = cv2.imread("aia_logo.png")[:, :, ::-1]

In [ ]:
plt.imshow(image)
plt.axis("off")
plt.show()

* ### 影像二值化
 -- 基本上影像都是用二進位表示的，最簡單的二進位表示方式就是將一個像素值用 0(黑) 或 1(白) 表示。<br>
 -- 而將一張灰階影像轉換成黑白值可以透過 cv2.threshold 來達成。<br>
 -- 此函數的作法是給其一個門檻值，小於門檻值的皆設為黑色，反之設為白色。<br>

In [ ]:
gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [ ]:
#  小於 127 設為黑
_, thresh1 = cv2.threshold(gray_img, 127, 1, cv2.THRESH_BINARY)

#  小於 200 設為黑
_, thresh2 = cv2.threshold(gray_img, 200, 1, cv2.THRESH_BINARY)


img_list = [gray_img, thresh1, thresh2]
title = ['Gray Image', 'Threshold: 127', 'Threshold: 200']

plt.figure(figsize=(12, 12))
for i, each in enumerate(img_list):
  plt.subplot(1, 3, i+1)
  plt.imshow(each, cmap='gray')
  plt.title(title[i], fontsize=15)
  plt.axis("off")
plt.show()

- 設定門檻值，讓灰階可以黑白分明，以下為各種不同函式的表現，可以觀察原圖與所使用的函式差異。

* #### Global Thresholding

In [ ]:
# upload bw.jpg
!gdown --id 1rtwidQ_IuoCNaykt9IdWuUyoE2026VRh

In [ ]:
image = cv2.imread('bw.jpg', cv2.IMREAD_GRAYSCALE)
(h, w) = image.shape[:2]
center = (w/2, h/2)
M = cv2.getRotationMatrix2D(center, 270, 2.0)
rotated = cv2.warpAffine(image, M, (w, h))
img = rotated

_, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
_, thresh2 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
_, thresh3 = cv2.threshold(img, 127, 255, cv2.THRESH_TRUNC)
_, thresh4 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO)
_, thresh5 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO_INV)

titles = ['Original Image', 'BINARY', 'BINARY_INV',
      'TRUNC', 'TOZERO', 'TOZERO_INV']
images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]

plt.figure(figsize=(12, 6))
for i in range(6):
  plt.subplot(2, 3, i+1), plt.imshow(images[i], 'gray')
  plt.title(titles[i])
  plt.axis("off")
plt.show()

[(back...)](#The-Advanced-of-Image-Preprocessing)

* #### Adaptive Thresholding


In [ ]:
# upload bicycle.jpg
!gdown --id 1nyJIv5Wu4jHwxVvsUqeSzYee_QDhmPTV

In [ ]:
image = cv2.imread('bicycle.jpg', cv2.IMREAD_GRAYSCALE)

# 二值化(未模糊降噪)
ret, th1 = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# 自適應平均二值化(未模糊降噪)
th2 = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
        
# 自適應高斯二值化(未模糊降噪)
th3 = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
      'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [image, th1, th2, th3]

for i in range(4):
  plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
  plt.title(titles[i])
  plt.xticks([]),plt.yticks([])
plt.show()

In [ ]:
image = cv2.imread('bicycle.jpg', cv2.IMREAD_GRAYSCALE)

# 將圖片做模糊化，可以降噪
blur_img = cv2.medianBlur(image, 5)
        
# 二值化(有模糊降噪)
ret, th4 = cv2.threshold(blur_img, 127, 255, cv2.THRESH_BINARY)

# 算術平均法的自適應二值化(有模糊降噪)
th5 = cv2.adaptiveThreshold(blur_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
        
# 高斯加權均值法自適應二值化(有模糊降噪)      
th6 = cv2.adaptiveThreshold(blur_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

titles = ['Blur Image', 'Global Thresholding (v = 127)',
      'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

for i in range(4):
  plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
  plt.title(titles[i])
  plt.xticks([]),plt.yticks([])
plt.show()

* ### 用 cv2.findContours 找影像輪廓

In [ ]:
# upload poker.jpg
!gdown --id 1jjsEi6UGrZ5YYvgMFJ8itmqvzyGvd50w

In [ ]:
image = cv2.imread('poker.jpg')[:, :, ::-1]
gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
new_image = image.copy()

# threshold image
ret, thresh = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)
# find contours and get the external one
contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


cv2.drawContours(new_image, contours, -1, (255, 255, 0), 3)

titles = ['Original_image', 'Threshed_image', 'DrawContours']
images = [image, thresh, new_image]


plt.figure(figsize=(18, 9))
for i in range(3):
  plt.subplot(2, 3, i+1), plt.imshow(images[i], 'gray')
  plt.title(titles[i])
  plt.axis("off")
plt.show()

In [ ]:
img = cv2.imread('poker.jpg', cv2.IMREAD_UNCHANGED)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
for i in range(4):
  for a in range(2):
    var_name = f'img_{i+1}{a+1}'
    locals()[var_name] =\
      cv2.imread('poker.jpg', cv2.IMREAD_UNCHANGED)
    locals()[var_name] =\
      cv2.cvtColor(locals()[var_name], cv2.COLOR_BGR2RGB)

img_g = cv2.GaussianBlur(img, (15, 15), 5)

for a in range(2):
  # 對影像做二值化處理
  if a == 0:
    ret, threshed_img = cv2.threshold(
        cv2.cvtColor(img_11, cv2.COLOR_RGB2GRAY),
        127, 255, cv2.THRESH_BINARY
    )
  if a == 1:
    ret, threshed_img = cv2.threshold(
        cv2.cvtColor(img_g, cv2.COLOR_RGB2GRAY),
        127, 255, cv2.THRESH_BINARY
    )

  # 找出二值化後的邊界
  contours, hier = cv2.findContours(threshed_img,
                     cv2.RETR_TREE,
                     cv2.CHAIN_APPROX_SIMPLE)

  # 沿著邊界找到小適合設定的定界框
  # 黃：只沿著邊界畫框的
  # 綠：沿著邊界畫出無旋轉角度的最小矩形
  # 藍：沿著邊界畫出有旋轉角度的最小矩形
  # 紅：沿著邊界畫出的最小圓形

  for c in contours:
    # get the bounding rect
    x, y, w, h = cv2.boundingRect(c)

    # 第一組圖
    # draw a green rectangle to visualize the bounding rect
    cv2.rectangle(locals()[f'img_1{a+1}'], (x, y),
               (x+w, y+h), (0, 255, 0), 3)

    # 取最小包含物體的區域
    rect = cv2.minAreaRect(c)
    box = cv2.boxPoints(rect)

    # 將值從 float 轉為 int
    box = np.int0(box)

    # 第二組圖
    # draw a red 'nghien' rectangle
    cv2.drawContours(locals()[f'img_2{a+1}'], [box],
             0, (0, 0, 255), 3)

    (x, y), radius = cv2.minEnclosingCircle(c)

    center = (int(x), int(y))
    radius = int(radius)

    # 第三組圖
    # and draw the circle in blue
    cv2.circle(locals()['img_3{}'.format(a+1)],
          center, radius, (255, 0, 0), 3)
  # 第四組圖
  # and draw the contour with yellow
  cv2.drawContours(locals()['img_4{}'.format(a+1)],
           contours, -1, (255, 255, 0), 3)


titles = ['Box', 'Box_blur', 'Box_2', 'Box_2_blur',
      'Circle', 'Circle_blur', 'Contours', 'Contours_blur']
images = [img_11, img_12, img_21, img_22,
      img_31, img_32, img_41, img_42]
plt.figure(figsize=(20, 20))
for i in range(8):
  plt.subplot(4, 2, i+1), plt.imshow(images[i])
  plt.title(titles[i])
  plt.axis("off")
plt.show()

[(back...)](#The-Advanced-of-Image-Preprocessing)

* ## K-Means Clustering in OpenCV
一般的影像儲存方式是三個通道 (RGB)，每個通道都是 0~255(8 bits)<br>
這樣一個像素需要花 8*3=24 bits 儲存<br>
若想要降低影像的儲存空間，K-Means 是一種很棒的方式<br>
我們可以將影像中的所有顏色做分群後，用群的中心點來代表這群所有點的值<br>
將影像的顏色分成 $2^5=32$ 個群，只需要花 5 bits 就可以代表一個像素了<br>
相對原本的儲存方式大約節省了五倍的記憶體容量了

In [ ]:
# upload flower2.jpg
!gdown --id 1NtTqw-W9fcwdgq0MyjA1wc3O4rkciqRH

In [ ]:
img = cv2.imread('flower2.jpg')[:, :, ::-1]
Z = img.reshape((-1, 3))

# convert to np.float32
Z = np.float32(Z)

# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

result = []
title = []

# [不分群, 分 2 群, 分 4 群, 分 8 群, 分 16 群, 分 32 群]
for i in range(1, 6):
  K = 2 ** i
  ret, label, center = cv2.kmeans(Z, K, None,
                    criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

  center = np.uint8(center)
  cluster = center[label.flatten()]
  cluster = cluster.reshape((img.shape))

  title .append(str(K)+"-clustering")
  result.append(cluster)


titles = ['Original_img'] + title
images = [img] + result

plt.figure(figsize=(16, 8))
for i in range(6):
  plt.subplot(2, 3, i+1)
  plt.imshow(images[i])
  plt.title(titles[i])
  plt.axis("off")
plt.show()

[(back...)](#The-Advanced-of-Image-Preprocessing)